In [1]:
# !pip install autogluon

In [2]:
from autogluon.tabular import TabularDataset, TabularPredictor

In [3]:
import pandas as pd

In [4]:
from autogluon.tabular import TabularDataset, TabularPredictor
from autogluon.common.utils.utils import setup_outputdir
from autogluon.core.utils.loaders import load_pkl
from autogluon.core.utils.savers import save_pkl
import os.path

class MultilabelPredictor():
    """ Tabular Predictor for predicting multiple columns in table.
        Creates multiple TabularPredictor objects which you can also use individually.
        You can access the TabularPredictor for a particular label via: `multilabel_predictor.get_predictor(label_i)`

        Parameters
        ----------
        labels : List[str]
            The ith element of this list is the column (i.e. `label`) predicted by the ith TabularPredictor stored in this object.
        path : str, default = None
            Path to directory where models and intermediate outputs should be saved.
            If unspecified, a time-stamped folder called "AutogluonModels/ag-[TIMESTAMP]" will be created in the working directory to store all models.
            Note: To call `fit()` twice and save all results of each fit, you must specify different `path` locations or don't specify `path` at all.
            Otherwise files from first `fit()` will be overwritten by second `fit()`.
            Caution: when predicting many labels, this directory may grow large as it needs to store many TabularPredictors.
        problem_types : List[str], default = None
            The ith element is the `problem_type` for the ith TabularPredictor stored in this object.
        eval_metrics : List[str], default = None
            The ith element is the `eval_metric` for the ith TabularPredictor stored in this object.
        consider_labels_correlation : bool, default = True
            Whether the predictions of multiple labels should account for label correlations or predict each label independently of the others.
            If True, the ordering of `labels` may affect resulting accuracy as each label is predicted conditional on the previous labels appearing earlier in this list (i.e. in an auto-regressive fashion).
            Set to False if during inference you may want to individually use just the ith TabularPredictor without predicting all the other labels.
        kwargs :
            Arguments passed into the initialization of each TabularPredictor.

    """

    multi_predictor_file = 'multilabel_predictor.pkl'

    def __init__(self, labels, path=None, problem_types=None, eval_metrics=None, consider_labels_correlation=True, **kwargs):
        if len(labels) < 2:
            raise ValueError("MultilabelPredictor is only intended for predicting MULTIPLE labels (columns), use TabularPredictor for predicting one label (column).")
        if (problem_types is not None) and (len(problem_types) != len(labels)):
            raise ValueError("If provided, `problem_types` must have same length as `labels`")
        if (eval_metrics is not None) and (len(eval_metrics) != len(labels)):
            raise ValueError("If provided, `eval_metrics` must have same length as `labels`")
        self.path = setup_outputdir(path, warn_if_exist=False)
        self.labels = labels
        self.consider_labels_correlation = consider_labels_correlation
        self.predictors = {}  # key = label, value = TabularPredictor or str path to the TabularPredictor for this label
        if eval_metrics is None:
            self.eval_metrics = {}
        else:
            self.eval_metrics = {labels[i] : eval_metrics[i] for i in range(len(labels))}
        problem_type = None
        eval_metric = None
        for i in range(len(labels)):
            label = labels[i]
            path_i = self.path + "Predictor_" + label
            if problem_types is not None:
                problem_type = problem_types[i]
            if eval_metrics is not None:
                eval_metric = eval_metrics[i]
            self.predictors[label] = TabularPredictor(label=label, problem_type=problem_type, eval_metric=eval_metric, path=path_i, **kwargs)

    def fit(self, train_data, tuning_data=None, **kwargs):
        """ Fits a separate TabularPredictor to predict each of the labels.

            Parameters
            ----------
            train_data, tuning_data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                See documentation for `TabularPredictor.fit()`.
            kwargs :
                Arguments passed into the `fit()` call for each TabularPredictor.
        """
        if isinstance(train_data, str):
            train_data = TabularDataset(train_data)
        if tuning_data is not None and isinstance(tuning_data, str):
            tuning_data = TabularDataset(tuning_data)
        train_data_og = train_data.copy()
        if tuning_data is not None:
            tuning_data_og = tuning_data.copy()
        else:
            tuning_data_og = None
        save_metrics = len(self.eval_metrics) == 0
        for i in range(len(self.labels)):
            label = self.labels[i]
            predictor = self.get_predictor(label)
            if not self.consider_labels_correlation:
                labels_to_drop = [l for l in self.labels if l != label]
            else:
                labels_to_drop = [self.labels[j] for j in range(i+1, len(self.labels))]
            train_data = train_data_og.drop(labels_to_drop, axis=1)
            if tuning_data is not None:
                tuning_data = tuning_data_og.drop(labels_to_drop, axis=1)
            print(f"Fitting TabularPredictor for label: {label} ...")
            predictor.fit(train_data=train_data, tuning_data=tuning_data, **kwargs)
            self.predictors[label] = predictor.path
            if save_metrics:
                self.eval_metrics[label] = predictor.eval_metric
        self.save()

    def predict(self, data, **kwargs):
        """ Returns DataFrame with label columns containing predictions for each label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. If label columns are present in this data, they will be ignored. See documentation for `TabularPredictor.predict()`.
            kwargs :
                Arguments passed into the predict() call for each TabularPredictor.
        """
        return self._predict(data, as_proba=False, **kwargs)

    def predict_proba(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `predict_proba()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to make predictions for. See documentation for `TabularPredictor.predict()` and `TabularPredictor.predict_proba()`.
            kwargs :
                Arguments passed into the `predict_proba()` call for each TabularPredictor (also passed into a `predict()` call).
        """
        return self._predict(data, as_proba=True, **kwargs)

    def evaluate(self, data, **kwargs):
        """ Returns dict where each key is a label and the corresponding value is the `evaluate()` output for just that label.

            Parameters
            ----------
            data : str or autogluon.tabular.TabularDataset or pd.DataFrame
                Data to evalate predictions of all labels for, must contain all labels as columns. See documentation for `TabularPredictor.evaluate()`.
            kwargs :
                Arguments passed into the `evaluate()` call for each TabularPredictor (also passed into the `predict()` call).
        """
        data = self._get_data(data)
        eval_dict = {}
        for label in self.labels:
            print(f"Evaluating TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            eval_dict[label] = predictor.evaluate(data, **kwargs)
            if self.consider_labels_correlation:
                data[label] = predictor.predict(data, **kwargs)
        return eval_dict

    def save(self):
        """ Save MultilabelPredictor to disk. """
        for label in self.labels:
            if not isinstance(self.predictors[label], str):
                self.predictors[label] = self.predictors[label].path
        save_pkl.save(path=self.path+self.multi_predictor_file, object=self)
        print(f"MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('{self.path}')")

    @classmethod
    def load(cls, path):
        """ Load MultilabelPredictor from disk `path` previously specified when creating this MultilabelPredictor. """
        path = os.path.expanduser(path)
        if path[-1] != os.path.sep:
            path = path + os.path.sep
        return load_pkl.load(path=path+cls.multi_predictor_file)

    def get_predictor(self, label):
        """ Returns TabularPredictor which is used to predict this label. """
        predictor = self.predictors[label]
        if isinstance(predictor, str):
            return TabularPredictor.load(path=predictor)
        return predictor

    def _get_data(self, data):
        if isinstance(data, str):
            return TabularDataset(data)
        return data.copy()

    def _predict(self, data, as_proba=False, **kwargs):
        data = self._get_data(data)
        if as_proba:
            predproba_dict = {}
        for label in self.labels:
            print(f"Predicting with TabularPredictor for label: {label} ...")
            predictor = self.get_predictor(label)
            if as_proba:
                predproba_dict[label] = predictor.predict_proba(data, as_multiclass=True, **kwargs)
            data[label] = predictor.predict(data, **kwargs)
        if not as_proba:
            return data[self.labels]
        else:
            return predproba_dict

In [5]:
!unzip '/content/신약.zip'

train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
train_data = train.drop(columns=['id'])
test_data = test.drop(columns=['id'])

Archive:  /content/신약.zip
  inflating: sample_submission.csv   
  inflating: test.csv                
  inflating: train.csv               


In [6]:
train.shape, train_data.shape, test.shape, test_data.shape

((3498, 11), (3498, 10), (483, 9), (483, 8))

In [7]:
!pip install rdkit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.7/29.7 MB 14.6 MB/s eta 0:00:00


In [8]:
import random
import os

import numpy as np
import pandas as pd

from sklearn.feature_selection import VarianceThreshold
from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

from rdkit import DataStructs
from rdkit.Chem import PandasTools, AllChem
PandasTools.AddMoleculeColumnToFrame(train,'SMILES','Molecule')
PandasTools.AddMoleculeColumnToFrame(test,'SMILES','Molecule')
def mol2fp(mol):
    fp = AllChem.GetHashedMorganFingerprint(mol, 6, nBits=4096)
    ar = np.zeros((1,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp, ar)
    return ar
# FPs column 추가
train["FPs"] = train.Molecule.apply(mol2fp)
test["FPs"] = test.Molecule.apply(mol2fp)

In [14]:
train.shape, test.shape

((3498, 13), (483, 11))

In [16]:
labels = ['MLM','HLM']  # which columns to predict based on the others
problem_types = ['regression','regression']  # type of each prediction problem (optional)
eval_metrics = ['mean_absolute_error','mean_absolute_error']  # metrics used to evaluate predictions for each label (optional)
save_path = 'agModels-predictEducationClass'  # specifies folder to store trained models (optional)

time_limit = 60*60*1  # how many seconds to train the TabularPredictor for each label, set much larger in your applications!

In [17]:
multi_predictor = MultilabelPredictor(labels=labels, problem_types=problem_types, eval_metrics=eval_metrics, path=save_path)
multi_predictor.fit(train_data, time_limit=time_limit,num_stack_levels=3,num_bag_folds=3,presets='best_quality',)

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=3, num_bag_folds=3, num_bag_sets=20
Beginning AutoGluon training ... Time limit = 3600s
AutoGluon will save models to "agModels-predictEducationClass/Predictor_MLM/"
AutoGluon Version:  0.8.2
Python Version:     3.10.12
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP Wed Aug 30 11:19:59 UTC 2023
Disk Space Avail:   80.69 GB / 115.66 GB (69.8%)
Train Data Rows:    3498
Train Data Columns: 8
Label Column: MLM
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    11390.64 MB
	Train Data (Original)  Memory Usage: 0.55 MB (0.0% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.


Fitting TabularPredictor for label: MLM ...


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', [])  : 4 | ['AlogP', 'Molecular_Weight', 'LogD', 'Molecular_PolarSurfaceArea']
		('int', [])    : 3 | ['Num_H_Acceptors', 'Num_H_Donors', 'Num_RotatableBonds']
		('object', []) : 1 | ['SMILES']
	Types of features in processed data (raw dtype, special dtypes):
		('category', []) : 1 | ['SMILES']
		('float', [])    : 4 | ['AlogP', 'Molecular_Weight', 'LogD', 'Molecular_PolarSurfaceArea']
		('int', [])      : 3 | ['Num_H_Acceptors', 'Num_H_Donors', 'Num_RotatableBonds']
	0.3s = Fit runtime
	8 features in original data used to gene

Fitting TabularPredictor for label: HLM ...


	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Stage 5 Generators:
		Fitting DropDuplicatesFeatureGenerator...
	Types of features in original data (raw dtype, special dtypes):
		('float', [])  : 5 | ['MLM', 'AlogP', 'Molecular_Weight', 'LogD', 'Molecular_PolarSurfaceArea']
		('int', [])    : 3 | ['Num_H_Acceptors', 'Num_H_Donors', 'Num_RotatableBonds']
		('object', []) : 1 | ['SMILES']
	Types of features in processed data (raw dtype, special dtypes):
		('category', []) : 1 | ['SMILES']
		('float', [])    : 5 | ['MLM', 'AlogP', 'Molecular_Weight', 'LogD', 'Molecular_PolarSurfaceArea']
		('int', [])      : 3 | ['Num_H_Acceptors', 'Num_H_Donors', 'Num_RotatableBonds']
	0.3s = Fit runtime
	9 features in original dat

MultilabelPredictor saved to disk. Load with: MultilabelPredictor.load('agModels-predictEducationClass/')


In [18]:
predictions = multi_predictor.predict(test_data)

Predicting with TabularPredictor for label: MLM ...
Predicting with TabularPredictor for label: HLM ...


In [19]:
predictions

,MLM,HLM
0,12.383641,33.208130
1,69.470642,90.377838
2,26.706392,59.240593
3,51.936188,80.258705
4,71.702370,88.863014
...,...,...
478,1.314675,6.005869
479,84.039719,94.314224
480,47.641994,66.374550
481,75.932823,83.140060


In [20]:
# predict
df_submission = pd.read_csv("./sample_submission.csv")
df_submission["MLM"] = predictions['MLM'].to_numpy()
df_submission["HLM"] = predictions['HLM'].to_numpy()
df_submission.to_csv("automl_result3.csv", index = False, encoding = "utf-8-sig")

In [ ]:

# df_submission.to_csv("/content/drive/MyDrive/신약개발/automl_result3.csv", index = False, encoding = "utf-8-sig")

앙상블

In [ ]:
# # 모델의 평가 지표 및 시간 설정
# eval_metrics = ['root_mean_squared_error', 'mean_absolute_error', 'r2']
# time_limits = [5400, 5400, 5400]
# model_paths = ['AutogluonModels/model_rmse_1h30m', 'AutogluonModels/model_mae_1h30m', 'AutogluonModels/model_r2_1h30m']

# all_predictions = []

# # 각 평가 지표로 모델 훈련 및 예측
# for i, (eval_metric, time_limit, save_path) in enumerate(zip(eval_metrics, time_limits, model_paths)):
#     multi_predictor = MultilabelPredictor(labels=labels, problem_types=problem_types, eval_metrics=eval_metrics, path=save_path)
#     multi_predictor.fit(train_data, time_limit=time_limit,num_stack_levels=3,num_bag_folds=3,presets='best_quality',)

#     test_pred = multi_predictor.predict(test_data)
#     all_predictions.append(test_pred)

#     # submission 파일 저장
#     submission = pd.read_csv('sample_submission.csv')
#     submission["MLM"] = predictions['MLM'].to_numpy()
#     submission["HLM"] = predictions['HLM'].to_numpy()
#     submission.to_csv(f'./autogluon_model_{eval_metric}_1h30m_submit.csv', index=False)

# # 모든 모델의 예측값을 앙상블
# ensemble_test_pred = sum(all_predictions) / len(all_predictions)
# # 앙상블 결과로 submission 파일 저장
